In [15]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr,wilcoxon
from statsmodels.sandbox.stats.multicomp import multipletests 
from itertools import combinations

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. 

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [4]:
clfs = pd.read_csv('AUCs.txt',sep='\t').rename({'Unnamed: 0':'Dataset'}, axis=1)
clfs.head()

,Dataset,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [14]:
combs = list(combinations(clfs.drop('Dataset',axis=1).columns,2))

clfs_diffs = []
for comb in combs:
    stat,p_value = wilcoxon(clfs[comb[0]],clfs[comb[1]])
    clfs_diffs.append([comb[0],comb[1],stat,p_value])

In [36]:
stat_table = pd.DataFrame(columns = ['clf1', 'clf2', 'stat', 'p_value'],
            data = clfs_diffs)
stat_table.sort_values(by='p_value',ascending=True)  

,clf1,clf2,stat,p_value
0,C4.5,C4.5+m,6.5,0.010757
2,C4.5,C4.5+m+cf,11.0,0.015906
5,C4.5+cf,C4.5+m+cf,10.0,0.022909
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
1,C4.5,C4.5+cf,43.0,0.861262


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [37]:
stat_table[stat_table['p_value']<0.05]['p_value'].count()

4

Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

In [38]:
stat_table[stat_table['p_value']<0.05]

,clf1,clf2,stat,p_value
0,C4.5,C4.5+m,6.5,0.010757
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
5,C4.5+cf,C4.5+m+cf,10.0,0.022909


Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [49]:
reject, p_corrected, a1, a2 = multipletests(stat_table['p_value'], 
                                            alpha = 0.05, 
                                            method = 'holm')
stat_table['p_corrected'] = p_corrected
stat_table['reject'] = reject

In [43]:
stat_table

,clf1,clf2,stat,p_value,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.091636,False


In [51]:
stat_table[stat_table.p_corrected < 0.05].reject.sum()

0

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [52]:
reject, p_corrected, a1, a2 = multipletests(stat_table['p_value'], 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')
stat_table['p_corrected'] = p_corrected
stat_table['reject'] = reject

In [47]:
stat_table

,clf1,clf2,stat,p_value,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.045818,True
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,17.0,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.045818,True


In [53]:
stat_table.reject.sum()

3